The following cell defines the functions needed for demonstrating the pretrained models

In [ ]:
#imports
import tensorflow as tf
import keras
from keras import layers, activations
from keras.initializers import RandomNormal
import matplotlib.pyplot as plt
import numpy as np

#format generator output for display
def format_image(generator_output):
    unscaled_image = generator_output[0,:,:,:]
    scaled_image = (generator_output*127.5)+127.5
    clipped_image = np.clip(scaled_image,0.0,255.0)
    clipped_image/=255.0
    return clipped_image[0]

#display generated images
def display_images(images):

    for idx in range(len(images)):
        img = format_image(images[idx])
        plt.imshow(img)
        plt.title('Test image #'+str(idx+1))
        plt.axis('off')
        plt.show()
        plt.clf()

#make generator following architecture outlined in paper
def make_generator_model():

    #input layer takes noise vector
    input_layer = keras.Input(shape=(100,))

    #fully connected layer without bias or activation to reshape input
    fcl = layers.Dense(4*4*3, use_bias=False,name='fcl')(input_layer)

    #reshape fully connected output
    reshape = layers.Reshape((4, 4, 3),name='reshape')(fcl)

    #transpose convolution block 1 using 512 filters, (mu=0,std=.02) normal initialization, batch normalization, and relu activation
    tconv1 = layers.Conv2DTranspose(512,3,padding='same',kernel_initializer=RandomNormal(mean=0.0, stddev=0.02),name='tconv1')(reshape)
    bnorm1 = layers.BatchNormalization()(tconv1)
    relu1 = layers.Activation(activations.relu)(bnorm1)

    #transpose convolution block 2 using 256 filters, (mu=0,std=.02) normal initialization, batch normalization, and relu activation
    tconv2 = layers.Conv2DTranspose(256,3,strides=2,padding='same',kernel_initializer=RandomNormal(mean=0.0, stddev=0.02),name='tconv2')(relu1)
    bnorm2 = layers.BatchNormalization()(tconv2)
    relu2 = layers.Activation(activations.relu)(bnorm2)

    #self attention layer
    attention = layers.MultiHeadAttention(1,256,kernel_initializer=RandomNormal(mean=0.0, stddev=0.02),name='attention')(relu2,relu2)

    #transpose convolution block 3 using 128 filters, (mu=0,std=.02) normal initialization, batch normalization, and relu activation
    tconv3 = layers.Conv2DTranspose(128,3,strides=2,padding='same',kernel_initializer=RandomNormal(mean=0.0, stddev=0.02),name='tconv3')(attention)
    bnorm3 = layers.BatchNormalization()(tconv3)
    relu3 = layers.Activation(activations.relu)(bnorm3)

    #transpose convolution block 4 using 64 filters, (mu=0,std=.02) normal initialization, batch normalization, and relu activation
    tconv4 = layers.Conv2DTranspose(64,3,strides=2,padding='same',kernel_initializer=RandomNormal(mean=0.0, stddev=0.02),name='tconv4')(tconv3)
    bnorm4 = layers.BatchNormalization()(tconv4)
    relu4 = layers.Activation(activations.relu)(bnorm4)

    #transpose convolution block 5 (output layer) using 3 filters, (mu=0,std=.02) normal initialization, batch normalization, and relu activation
    output_layer = layers.Conv2DTranspose(3,3,strides=2,padding='same',kernel_initializer=RandomNormal(mean=0.0, stddev=0.02),activation='tanh',name='output')(relu4)

    #assemble model
    model = keras.Model(inputs=input_layer, outputs=output_layer,name='generator')

    return model

#make discriminator following architecture outlined in paper
def make_discriminator_model():

    #input layer taking output from discriminator
    input_layer = keras.Input(shape=[64, 64, 3])

    #spectral normalization layer with 64 filters and leaky relu activation
    spec_norm = layers.SpectralNormalization(layers.Conv2D(64,3,strides=2,padding='same',name='spec_norm'))(input_layer)
    lrelu1 = layers.LeakyReLU(name='lrelu1')(spec_norm)

    #convolutional block 1 with 128 filters and leaky relu activation
    conv1 = layers.Conv2D(128,3,strides=2,padding='same',name='conv1')(lrelu1)
    bnorm1 = layers.BatchNormalization()(conv1)
    lrelu2 = layers.LeakyReLU(name='lrelu2')(bnorm1)

    #convolutional block 2 with 256 filters and leaky relu activation
    conv2 = layers.Conv2D(256,3,strides=2,padding='same',name='conv2')(lrelu2)
    bnorm2 = layers.BatchNormalization()(conv2)
    lrelu3 = layers.LeakyReLU(name='lrelu3')(conv2)

    #convolutional block 3 with 512 filters and leaky relu activation
    conv3 = layers.Conv2D(512,3,strides=2,padding='same',name='conv3')(lrelu3)
    bnorm3 = layers.BatchNormalization()(conv3)
    lrelu4 = layers.LeakyReLU(name='lrelu4')(conv3)

    #flatten convolutional block output
    flattened = layers.Flatten()(lrelu4)

    #spectral normalization output layer with (mu=0,std=.02) normal initialization, one unit and sigmoid activation (binary classification for real vs fake)
    output_layer = layers.SpectralNormalization(layers.Dense(1,kernel_initializer=RandomNormal(mean=0.0, stddev=0.02),activation='sigmoid',name='output_layer'))(flattened)

    #assemble model
    model = keras.Model(inputs=input_layer,outputs=output_layer,name='discriminator')

    return model

The following cell allows using the model trained on the Anime Faces dataset to generate sample images

In [ ]:
#please change to the path of the animefacedataset generator weights 
animefacedataset_generator_weights_path = 'animefacedataset_generator.weights.h5'
#please change to the path of the animefacedataset discriminator weights
animefacedataset_discriminator_weights_path = 'animefacedataset_discriminator.weights.h5'  

#make generator
animefacedataset_generator = make_generator_model()
#make discriminator
animefacedataset_discriminator = make_discriminator_model()

#load generator weights
animefacedataset_generator.load_weights(animefacedataset_generator_weights_path)
#load discriminator weights
animefacedataset_discriminator.load_weights(animefacedataset_discriminator_weights_path)

#number of images to generate - please change to desired number
animefacedataset_NUM_TEST_IMAGES = 4
animefacedataset_test_images = []

#generate images
for idx in range(animefacedataset_NUM_TEST_IMAGES):
    noise = tf.random.normal([1, 100])
    generated_image = animefacedataset_generator(noise, training=False)
    animefacedataset_test_images.append(generated_image)

#display images
display_images(animefacedataset_test_images)

The following cell allows using the model trained on the celebA dataset to generate sample images

In [ ]:
#please change to the path of the celebA generator weights 
celebA_generator_weights_path = 'celebA_generator.weights.h5'
#please change to the path of the celebA discriminator weights
celebA_discriminator_weights_path = 'celebA_discriminator.weights.h5'  

#make generator
celebA_generator = make_generator_model()
#make discriminator
celebA_discriminator = make_discriminator_model()

#load generator weights
celebA_generator.load_weights(celebA_generator_weights_path)
#load discriminator weights
celebA_discriminator.load_weights(celebA_discriminator_weights_path)

#number of images to generate - please change to desired number
celebA_NUM_TEST_IMAGES = 4
celebA_test_images = []

#generate images
for idx in range(celebA_NUM_TEST_IMAGES):
    noise = tf.random.normal([1, 100])
    generated_image = celebA_generator(noise, training=False)
    celebA_test_images.append(generated_image)

#display images
display_images(celebA_test_images)

The following cell allows using the model trained on the iCartoonFace dataset to generate sample images

In [ ]:
#please change to the path of the iCartoonFace generator weights 
iCartoonFace_generator_weights_path = 'iCartoonFace_generator.weights.h5'
#please change to the path of the iCartoonFace discriminator weights
iCartoonFace_discriminator_weights_path = 'iCartoonFace_discriminator.weights.h5'  

#make generator
iCartoonFace_generator = make_generator_model()
#make discriminator
iCartoonFace_discriminator = make_discriminator_model()

#load generator weights
iCartoonFace_generator.load_weights(iCartoonFace_generator_weights_path)
#load discriminator weights
iCartoonFace_discriminator.load_weights(iCartoonFace_discriminator_weights_path)

#number of images to generate - please change to desired number
iCartoonFace_NUM_TEST_IMAGES = 4
iCartoonFace_test_images = []

#generate images
for idx in range(iCartoonFace_NUM_TEST_IMAGES):
    noise = tf.random.normal([1, 100])
    generated_image = iCartoonFace_generator(noise, training=False)
    iCartoonFace_test_images.append(generated_image)

#display images
display_images(iCartoonFace_test_images)

The following cell allows using the model trained on the LFW dataset to generate sample images

In [ ]:
#please change to the path of the lfw_dataset generator weights 
lfw_dataset_generator_weights_path = 'lfw_dataset_generator.weights.h5'
#please change to the path of the lfw_dataset discriminator weights
lfw_dataset_discriminator_weights_path = 'lfw_dataset_discriminator.weights.h5'  

#make generator
lfw_dataset_generator = make_generator_model()
#make discriminator
lfw_dataset_discriminator = make_discriminator_model()

#load generator weights
lfw_dataset_generator.load_weights(lfw_dataset_generator_weights_path)
#load discriminator weights
lfw_dataset_discriminator.load_weights(lfw_dataset_discriminator_weights_path)

#number of images to generate - please change to desired number
lfw_dataset_NUM_TEST_IMAGES = 4
lfw_dataset_test_images = []

#generate images
for idx in range(lfw_dataset_NUM_TEST_IMAGES):
    noise = tf.random.normal([1, 100])
    generated_image = lfw_dataset_generator(noise, training=False)
    lfw_dataset_test_images.append(generated_image)

#display images
display_images(lfw_dataset_test_images)